# <center>Tutorial zaapy.api<center>

## <u>First step:</u>

- import zaapy.api </p>

This allows you to use the API version of the package

In [ ]:
from zaapy.api import GasDataSet
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

To load the data, we need to use the **GasDataSet** function.</p>
To work this function requires several arguments:
- timestep (always first argument)
- geometry
- directory
- wanted_keys (fields to load)</p>

Disclaimer: 
- the package works only following the GRZeltron format
- only supports .h5 files


In [ ]:
zaapy_path=Path("/home/soudaisa/github/zaapy/")
data_path=zaapy_path / "tests/grzeltron" # To complete where you cloned the github repo

## <u>Second step:</u> Loading the data

In [ ]:
ds = GasDataSet(0, geometry="spherical", directory=data_path,wanted_keys='electrons')


If you give a timestep that is not present in your data

In [ ]:
ds = GasDataSet(1, geometry="spherical", directory=data_path,wanted_keys='electrons')


If you mispell the wanted fields

In [ ]:
ds = GasDataSet(0, geometry="spherical", directory=data_path,wanted_keys='electron')


## <u>Third step:</u> Looking at the loaded data

In [ ]:
ds = GasDataSet(100, geometry="spherical", directory=data_path,wanted_keys='electrons')

To see the loaded fields:

In [ ]:
ds.keys()

If we load multiple fields:

In [ ]:
ds = GasDataSet(100, geometry="spherical", directory=data_path,wanted_keys='electrons,Bru,Dru')
ds.keys()

To access the parameters of the simulation:
- input_params.dat
- phys_params.dat

In [ ]:
ds.simu_params

Get a precise value:

In [ ]:
B0=ds.simu_params["B0"]
print(B0)

If you are missing a field for your analysis, you can simply update the dictionary **ds** 

In [ ]:
ds.update(GasDataSet(100, geometry="spherical", directory=data_path,wanted_keys='positrons'))


Check if the new field has been added

In [ ]:
ds.keys()

Additional information encapsulated in **ds**

In [ ]:
print("Directory = ",ds.directory)

In [ ]:
print("Geometry = ",ds.geometry)

In [ ]:
print(ds.dict)

In [ ]:
print("Timestep =",ds.it)

In [ ]:
print("Grid in r:", ds.grid.r)

In [ ]:
print("Grid in theta:", ds.grid.theta)

## <u>Fourth step</u>: Where are the data values? 

In [ ]:
ds["Bru"].data

In [ ]:
np.shape(ds["Bru"].data)

Use TAB to see the quantities available

In [ ]:
Bru=ds["Bru"]

In [ ]:
Bru.data

## Visualizing the data

There are two ways of visualizing the data:
- regular matplotlib
- map & plot function from zaapy

With matplotlib:

Give the right expression for **A** and **B** (tip: use TAB to navigate the available instances of Bru)

In [ ]:
A = # A is the r grid
B = # B is the theta grid

fig,ax=plt.subplots(subplot_kw=dict(polar=True))
cax=ax.pcolormesh(B[:-1],A[:-1],Bru.data.T/B0,vmin=-0.3,vmax=0.3,cmap="PuOr")
fig.colorbar(cax)
ax.set_rmin(-0.0001*np.min(A))
ax.set_rmax(np.max(A))
ax.set_thetamax(np.max(B)*180/np.pi)
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)

plt.show()

Field lines are stored in **.mfl** if available

In [ ]:
field_lines = # instance of Bru, for example data is an instance of Bru

fig,ax=plt.subplots(subplot_kw=dict(polar=True))
cax=ax.pcolormesh(B[:-1],A[:-1],Bru.data.T/B0,vmin=-0.3,vmax=0.3,cmap="PuOr")
fig.colorbar(cax)
ax.contour(B[:-1],A[:-1],field_lines.T/B0,levels=20,colors="k")
ax.set_rmin(-0.0001*np.min(A))
ax.set_rmax(np.max(A))
ax.set_thetamax(np.max(B)*180/np.pi)
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)

plt.show()

Using **map** and **plot** functions from zaapy

Directly from the dataset

In [ ]:
dsop = ds["Bru"].map("x", "z")
fig, ax = plt.subplots()
dsop.plot(fig, ax, cmap="PuOr", vmin=-1e4,vmax=1e4, title=r"$B_\phi$")
ax.set_aspect("equal")

plt.show()

Or direcly

In [ ]:
fig, ax = plt.subplots()
Bru.map("x","z").plot(fig, ax, cmap="PuOr", vmin=-0.3*B0,vmax=0.3*B0, title=r"$B_\phi$")
ax.set_aspect("equal")

plt.show()

Varying the number of levels (arg=levels) and removing the colorbar (arg=title)

In [ ]:
fig, ax = plt.subplots()
Bru.map("x","z").plot(fig, ax, cmap="PuOr", vmin=-0.3*B0,vmax=0.3*B0, levels=20)
ax.set_aspect("equal")

plt.show()

If **log(data)** needed, use argument **log=True**

In [ ]:
dsop = ds["electrons"].map("x", "z")
fig, ax = plt.subplots()
dsop.plot(fig, ax, log=True, cmap="viridis", vmin=1,vmax=3, title=r"$\rho_{\rm mid}$")
ax.set_aspect("equal")

plt.show()

## Computing new fields

To do so, we need to imporeet the **compute** function

In [ ]:
from zaapy.api import compute

Example of computation

In [ ]:
data_path=zaapy_path / "tests/half_domain/"
ds = GasDataSet(46000, geometry="spherical", directory=data_path,wanted_keys='Dru')
ds.keys()
B0=ds.simu_params["B0"]

In [ ]:
fig,ax=plt.subplots()
ds["Dru"].map("x","z").plot(fig,ax,vmin=-0.002*B0,vmax=0.002*B0,cmap="PuOr_r",title="Dru")
ax.set_aspect("equal")
plt.show()

Compute the absolute value of Dru

In [ ]:
abs_Dru = compute(
  field="|Dru|",
  data=np.abs(ds["Dru"].data),
  ref=ds["Dru"],
)

In [ ]:
fig,ax=plt.subplots()
abs_Dru.map("x","z").plot(fig,ax,vmin=-0.002*B0,vmax=0.002*B0,cmap="PuOr_r",title="abs(Dru)")
ax.set_aspect("equal")
plt.show()

Let's compute the pairs density

In [ ]:
ds = GasDataSet(46000, geometry="spherical", directory=data_path,wanted_keys='')
ds.keys()

In [ ]:
n0=

In [ ]:
pairs = compute(
  field=,
  data=,
  ref=,
)

In [ ]:
fig,ax=plt.subplots()
pairs.map("x","z").plot(fig,ax,cmap="viridis",vmin=,vmax=)
ax.set_aspect("equal")
plt.show()

In [ ]:
dsop = ds["electrons"].map("x", "z")
fig, ax = plt.subplots()
dsop.plot(fig, ax, cmap="viridis")
ax.set_aspect("equal")

plt.show()

# <center>Testing with numpy<center>

Use numpy as much as possible for faster scripts (numpy interfaces with C/C++)

In [ ]:
from numpy.testing import assert_array_almost_equal

In [ ]:
ds = GasDataSet(368000, geometry="spherical", directory="/home/soudaisa/charge_prod/reference/",wanted_keys='Bth,Bph,Eth,Eph')
Bth,Bph=ds["Bth"].data.T,ds["Bph"].data.T
Eth,Eph=ds["Eth"].data.T,ds["Eph"].data.T
dtheta=ds.grid.theta[1]-ds.grid.theta[0]

In [ ]:
theta=ds.grid.theta[:-1]
r=ds.grid.r[:-1]

In [ ]:
diss = np.einsum("ij,j,i->i", (Eth * Bph - Eph * Bth) * dtheta, np.sin(theta), 2.9979246e10 / 2.0 * r**2.0)


In [ ]:
import scipy
diss_loop = np.empty(len(r))
ddiss = np.empty(len(theta))
for ir in range(len(r)):
    for ith in range(len(theta)):
        ddiss[ith] = (
         2.9979246e10
         * (Eth[ir, ith] * Bph[ir, ith] - Eph[ir, ith] * Bth[ir, ith])
         / 2.0
         * r[ir]
         * r[ir]
         * np.sin(theta[ith])
        )

    diss_loop[ir] = scipy.integrate.simps(ddiss, x=theta)
print("Done")

In [ ]:
diss0 = 2.9979246e10 / 4.0 * ds.simu_params["B0 [G]"]**2 * ds.simu_params["rmin [cm]"]**6 / (ds.simu_params["rlc [cm]"]**4)
fig,ax=plt.subplots()
ax.plot(r,diss/diss0)
ax.plot(r,diss_loop/diss0)
plt.show()

In [ ]:
assert_array_almost_equal(diss/diss0,diss_loop/diss0,decimal=4)

In [ ]:
assert_array_almost_equal(diss/diss0,diss_loop/diss0,decimal=5)

In [ ]:
assert_array_almost_equal(diss/diss0,diss_loop/diss0,decimal=6)